## Using map() with a Basic Thread Pool

In [3]:
from concurrent import futures
import threading
import time

def task(n):
    print(f"{threading.current_thread().name} sleeping {n}")
    time.sleep(n)
    print(f"{threading.current_thread().name} done with {n}")
    return n/10

ex = futures.ThreadPoolExecutor(max_workers=2)
print("main:starting")
results = ex.map(task, range(5,0,-1))
print("main:unprocessed results", results)
print("main:waiting for real results")
real_res = list(results)
print("main:real results", real_res)

main:starting
ThreadPoolExecutor-1_0 sleeping 5
ThreadPoolExecutor-1_1 sleeping 4
main:unprocessed results <generator object Executor.map.<locals>.result_iterator at 0x1121db9e8>
main:waiting for real results
ThreadPoolExecutor-1_1 done with 4
ThreadPoolExecutor-1_1 sleeping 3
ThreadPoolExecutor-1_0 done with 5
ThreadPoolExecutor-1_0 sleeping 2
ThreadPoolExecutor-1_0 done with 2
ThreadPoolExecutor-1_0 sleeping 1
ThreadPoolExecutor-1_1 done with 3
ThreadPoolExecutor-1_0 done with 1
main:real results [0.5, 0.4, 0.3, 0.2, 0.1]


## Scheduling Individual Tasks

In [4]:
ex = futures.ThreadPoolExecutor(max_workers=2)
print("main:starting")
f = ex.submit(task, 5)
print("main:future:", f)
print("main:waiting for real results")
real_res = f.result()
print("main:real results", real_res)
print("main:future after results", f)

main:starting
ThreadPoolExecutor-2_0 sleeping 5
main:future: <Future at 0x11218c048 state=running>
main:waiting for real results
ThreadPoolExecutor-2_0 done with 5
main:real results 0.5
main:future after results <Future at 0x11218c048 state=finished returned float>


## Waiting for Tasks in Any Order

In [5]:
import random
def task(n):
    time.sleep(random.random())
    return (n, n/10)
    
ex = futures.ThreadPoolExecutor(max_workers=5)

wait_for = [ex.submit(task, i) for i in range(5,0,-1)]
for f in futures.as_completed(wait_for):
    print('main results :', f.result())

main results : (5, 0.5)
main results : (1, 0.1)
main results : (3, 0.3)
main results : (4, 0.4)
main results : (2, 0.2)


## Future callbacks

In [8]:
def task(n):
    print(f"{n}: sleeping ")
    time.sleep(0.5)
    print(f"{n}: done ")
    return n**3

def done(fn):
    if fn.cancelled():
        print(f"{fn.arg}: cancelled")
    elif fn.done():
        error = fn.exception()
        if error:
            print(f"{fn.arg}: error returned:{error}")
        else:
            result = fn.result()
            print(f"{fn.arg}: value returned:{result}")
ex = futures.ThreadPoolExecutor(max_workers=2)
print('main starting')
f = ex.submit(task, 5)
f.arg = 5
f.add_done_callback(done)
result = f.result()

main starting
5: sleeping 
5: done 
5: value returned:125


## Cancelling Tasks

In [10]:
ex = futures.ThreadPoolExecutor(max_workers=4)
print('main starting')
tasks = []
for i in range(10, 0, -1):
    print('main submiting ', i)
    f = ex.submit(task, i)
    f.arg = f"<arg{i}>"
    f.add_done_callback(done)
    tasks.append((i, f))
    
for i, t in reversed(tasks):
    if not t.cancel():
        print('main did not cancel ', i)
ex.shutdown()

main starting
main submiting  10
10: sleeping 
main submiting  9
9: sleeping main submiting  8

8: sleeping 
main submiting  7
7: sleeping 
main submiting  6
main submiting  5
main submiting  4
main submiting  3
main submiting  2
main submiting  1
<arg1>: cancelled
<arg2>: cancelled
<arg3>: cancelled
<arg4>: cancelled
<arg5>: cancelled
<arg6>: cancelled
main did not cancel  7
main did not cancel  8
main did not cancel  9
main did not cancel  10
10: done 
<arg10>: value returned:1000
8: done 9: done 
<arg9>: value returned:729

<arg8>: value returned:512
7: done 
<arg7>: value returned:343


## Exceptions in Tasks

In [11]:
def task(n):
    print(f'{n} starting')
    raise ValueError(f'the value {n} is not good')
    
ex = futures.ThreadPoolExecutor(max_workers=2)
print('main starting')
f = ex.submit(task, 5)
error = f.exception()
print('main error ', error)
try:
    result = f.result()
except ValueError as e:
    print('main saw error when accessing result ', e)

main starting
5 starting
main error  the value 5 is not good
main saw error when accessing result  the value 5 is not good


## Context Manager

In [12]:
def task(n):
    print(n)
    
with futures.ThreadPoolExecutor(max_workers=2) as ex:
    print('main starting')
    ex.submit(task, 1)
    ex.submit(task, 2)
    ex.submit(task, 3)
print('main done')

main starting
1
2
3
main done


## Process Pools

In [13]:
import os

def task(n):
    return (n, os.getpid())

ex = futures.ProcessPoolExecutor(max_workers=2)
results = ex.map(task , range(5,0,-1))
for n, pid in results:
    print(f'ran task{n} in {pid}')

ran task5 in 33242
ran task4 in 33243
ran task3 in 33242
ran task2 in 33242
ran task1 in 33242


In [14]:
import signal

with futures.ProcessPoolExecutor(max_workers=2) as ex:
    print('getting the pid for one worker')
    f1 = ex.submit(os.getpid)
    pid1 = f1.result()
    
    print('killing process ', pid1)
    os.kill(pid1, signal.SIGHUP)
    
    print('submiting another task')
    f2 = ex.submit(os.getpid)
    try:
        pid2 = f2.result()
    except futures.process.BrokenProcessPool as e:
        print('could not start new tasks ', e)

getting the pid for one worker
killing process  33352
submiting another task
could not start new tasks  A process in the process pool was terminated abruptly while the future was running or pending.
